In [ ]:
# Using the pipeline function from huggingface 

In [ ]:
from transformers import pipeline ,YolosImageProcessor, YolosForObjectDetection
from PIL import Image,ImageDraw, ImageFont
import requests
from IPython.display import display
import cv2 
import numpy as np
import sys
import os
import torch

# Loads the model ( model = pipeline(model="hustvl/yolos-tiny", device="cuda:0" for gpu usage)
model = pipeline(model="hustvl/yolos-tiny")


In [ ]:
# Function to detect humans in static images 

In [ ]:
# Loads the images
def load_images_from_folder(folder_path):
    images = []
    file_name = []
    # Goes through the folder
    for filename in os.listdir(folder_path):
        # If the file is an image
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Add any other image file extensions you need
            image_path = os.path.join(folder_path, filename)
            try:
                image = Image.open(image_path)
                # Adds the image to the images list 
                images.append(image)
                # Records the file-name loaded
                file_name.append(filename)
            except Exception as e:
                print(f"Failed to load image {filename}: {e}")
    return images, file_name

# Split list into chunks to ensure processor doesnt crash
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]
# Passes the image through the model for identification and classification of objects
def processess_image(loaded_images,file_name):
    # Process images in batches
    chunk_size = 50
    index = 0
    # Loops for every 50 images
    for i, image_chunk in enumerate(chunk_list(loaded_images, chunk_size)):
        print(f"Processing batch {i + 1}")
        # Sends 50 images to the AI model to process
        detections = model(image_chunk)
        # For each image
        for j in range(len(image_chunk)):       
            success = False
            while index < len(file_name):
                # Determines the original image path
                filename = file_name[index]
                image_path = os.path.join(folder_path, filename)
                if os.path.isdir(image_path):  # Skip directories
                    index += 1
                    continue
                image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                # Loops through labels
                for detection in detections[j]:
                    # If a person is detected in the image, record image as success
                    if detection['score'] > 0.9 and detection['label'] == "person":
                        success = True
                        break
                # Outputs the image to either human or nothuman directory 
                if not success:
                    output_path = f'nothuman/{filename}'
                else:
                    output_path = f'human/{filename}'
                outcome = cv2.imwrite(output_path, image)
                index += 1
                break
    print("Processing complete.")
    return


In [ ]:
# MAIN CODE 
# Ensure output directories exist
if not os.path.exists('human'):
    # Makes directories if it doesn't exist 
    os.makedirs('human')
if not os.path.exists('nothuman'):
    os.makedirs('nothuman')
# Folder path of the images
folder_path = '/workspaces/internship/images1'
# Loads the images to a list 
loaded_images,file_name = load_images_from_folder(folder_path)
# Processess the image
processess_image(loaded_images,file_name)


In [ ]:
# Loads images from big folders


In [ ]:
# Function to load images from a folder
def load_images_from_folder(folder_path, start_index, chunk_size):
    images = []
    file_names = []
    # List all files in the folder
    filenames = os.listdir(folder_path)
    # Loop through the specified chunk of files
    for filename in filenames[start_index:start_index + chunk_size]:
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            try:
                image = Image.open(image_path).convert('RGB')  # Ensure images are in RGB format
                images.append(image)
                file_names.append(filename)
            except Exception as e:
                print(f"Failed to load image {filename}: {e}")
    return images, file_names

# Ensure output directories exist
os.makedirs('human', exist_ok=True)
os.makedirs('nothuman', exist_ok=True)

# Folder path of the images
folder_path = '/home/jovyan/images2/images'

# Process images in batches of 100
chunk_size = 100
start_index = 0
total_files = len(os.listdir(folder_path))

while start_index < total_files:
    print(f"Loading and processing images from {start_index} to {start_index + chunk_size}")
    loaded_images, file_names = load_images_from_folder(folder_path, start_index, chunk_size)
    if not loaded_images:
        break

    detections = model(loaded_images)

    for i, image in enumerate(loaded_images):
        filename = file_names[i]
        image_path = os.path.join(folder_path, filename)
        image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)  # Convert PIL image to OpenCV format

        if image is None:
            print(f"Failed to read image {image_path}")
            continue

        success = False
        for detection in detections[i]:
            if detection['score'] > 0.9 and detection['label'] == "person":
                success = True
                break

        output_path = f'human/{filename}' if success else f'nothuman/{filename}'
        outcome = cv2.imwrite(output_path, image)

    start_index += chunk_size

print("Processing complete.")


In [ ]:
# Functions to detect vehicles in static images

In [ ]:
# Loads the images
def load_images_from_folder(folder_path):
    images = []
    file_name = []
    # Goes through the folder
    for filename in os.listdir(folder_path):
        # If the file is an image
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Add any other image file extensions you need
            image_path = os.path.join(folder_path, filename)
            try:
                image = Image.open(image_path)
                # Adds the image to the images list 
                images.append(image)
                # Records the file-name loaded
                file_name.append(filename)
            except Exception as e:
                print(f"Failed to load image {filename}: {e}")
    return images, file_name

# Split list into chunks to ensure processor doesnt crash
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# Ensure output directories exist
if not os.path.exists('vehicles'):
    # Makes directories if it doesn't exist 
    os.makedirs('vehicles')
if not os.path.exists('notvehicles'):
    os.makedirs('notvehicles')
# Folder path of the images
folder_path = '/workspaces/internship/images'
# Loads the images to a list 
loaded_images,file_name = load_images_from_folder(folder_path)

# Process images in batches
chunk_size = 50
index = 0
# Loops for every 50 images
for i, image_chunk in enumerate(chunk_list(loaded_images, chunk_size)):
    print(f"Processing batch {i + 1}")
    # Sends 50 images to the AI model to process
    detections = model(image_chunk)
    # For each image
    for j in range(len(image_chunk)):       
        success = False
        while index < len(file_name):
            # Determines the original image path
            filename = file_name[index]
            image_path = os.path.join(folder_path, filename)
            if os.path.isdir(image_path):  # Skip directories
                index += 1
                continue
            # Opens the image
            image = Image.open(image_path)
            img_array = np.asarray(image, dtype=np.uint8)
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            # Loops through labels
            for detection in detections[j]:
                # If a person is detected in the image, record image as success
                if detection['label'] in ["bus", "car", "motorcycle", "train", "truck"] and detection['score'] > 0.9:
                    success = True
                    break
            # Outputs the image to either human or nothuman directory 
            if not success:
                output_path = f'notvehicles/{filename}'
            else:
                output_path = f'vehicles/{filename}'
            outcome = cv2.imwrite(output_path, image)
            index += 1
            break
print("Processing complete.")


In [ ]:
# Check individual image for human

In [ ]:
from transformers import pipeline ,YolosImageProcessor, YolosForObjectDetection
from PIL import Image,ImageDraw, ImageFont
import requests
from IPython.display import display
import cv2 
import numpy as np
import sys
import os
import torch

# Loads the model 
model = pipeline(model="hustvl/yolos-tiny", device="cuda:0")


In [ ]:
image_path = os.path.join('/workspaces/internship/images','502.png')
image = Image.open(image_path)
detections = model(image_path)
print(detections)
# Converts the image into a Numpy Tensor 
img_array = np.asarray(image, dtype=np.uint8)
# Decodes and arranges the array into an array that can be processed 
image = cv2.imread(image_path, cv2.IMREAD_COLOR)
for detection in detections:
    if detection['score'] > 0.9:
        if detection['label'] == "person":
            # Extract box coordinates and label
            x1, y1, w, h = detection['box']['xmin'], detection['box']['ymin'], detection['box']['xmax'], detection['box']['ymax']
            label = "Human"
            score = f"{(detection['score'] *100):.1f}%"
            text = label + " " + score 

            # Draw the rectangle around the object
            cv2.rectangle(image, (x1, y1), (w, h), (0, 255, 0), 2)  # Green rectangle, thickness=2

            # Add the label text above the box
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, text , (x1, y1 - 10), font, 1.2, (0, 255, 0), 3)  # Green text
            print("Human")
else:
    print("Not Human")
        

# Renders image with labels and boxes and saves it.
output_path = 'sample.jpg'
outcome = cv2.imwrite(output_path,image)
image = Image.open(output_path)

In [ ]:
# Check individual image for vehicles

In [ ]:
image_path = os.path.join('/workspaces/internship/images','607.png')
image = Image.open(image_path)
detections = model(image_path)
# Converts the image into a Numpy Tensor 
img_array = np.asarray(image, dtype=np.uint8)
# Decodes and arranges the array into an array that can be processed 
image = cv2.imread(image_path, cv2.IMREAD_COLOR)
for detection in detections:
    if detection['score'] > 0.9:
        if detection['label'] in ["bus", "car", "motorcycle", "train", "truck"]:
            print(detection['score'])
            # Extract box coordinates and label
            x1, y1, w, h = detection['box']['xmin'], detection['box']['ymin'], detection['box']['xmax'], detection['box']['ymax']
            label = "Vehicle"
            score = f"{(detection['score'] *100):.1f}%"
            text = label + " " + score 

            # Draw the rectangle around the object
            cv2.rectangle(image, (x1, y1), (w, h), (0, 0, 0), 1)  # Green rectangle, thickness=2

            # Add the label text above the box
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, text , (x1, y1 - 10), font, 1.2, (0, 0, 0), 2)  # Green text
            print("Vehicle")
else:
    print("Not Vehicle")
        

# Renders image with labels and boxes and saves it.
output_path = 'sample.jpg'
outcome = cv2.imwrite(output_path,image)
image = Image.open(output_path)
image 